In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import MinMaxScaler

# Caminhos dos arquivos
CAMINHO_DADOS = r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\leads_novos_ficticios.csv"
CAMINHO_MODELOS = r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\modelos"

# Carregar dados e pré-processamento
df_leads = pd.read_csv(CAMINHO_DADOS)
df_leads["email"] = df_leads["email"].str.lower()
df_leads["lancamentos"] = "L2-25"

# Carregar artefatos do modelo
modelo_reg = joblib.load(f"{CAMINHO_MODELOS}/modelo_regressao_leadscore_total.pkl")
modelo_calibrado = joblib.load(f"{CAMINHO_MODELOS}/modelo_conversao_calibrado.pkl")
colunas_reg = joblib.load(f"{CAMINHO_MODELOS}/colunas_regressao.pkl")
colunas_calibrado = joblib.load(f"{CAMINHO_MODELOS}/colunas_modelo_conversao_calibrado.pkl")
limites = joblib.load(f"{CAMINHO_MODELOS}/limites_faixa.pkl")

# Função para classificar faixa pela média
def classificar_faixa_por_media(score):
    if score >= limites["limite_a"]:
        return "A"
    elif score >= limites["limite_b"]:
        return "B"
    elif score >= limites["limite_c"]:
        return "C"
    else:
        return "D"

# Preparar dados para regressão
X_reg = pd.get_dummies(df_leads, drop_first=False)
X_reg = X_reg.reindex(columns=colunas_reg, fill_value=0)

# Previsão do leadscore estimado
df_leads["leadscore_estimado_total"] = modelo_reg.predict(X_reg).round(2)
df_leads["faixa_predita_por_regressao"] = df_leads["leadscore_estimado_total"].apply(classificar_faixa_por_media)

# Preparar dados para probabilidade calibrada
X_calibrado = pd.get_dummies(df_leads, drop_first=False)
X_calibrado = X_calibrado.reindex(columns=colunas_calibrado, fill_value=0)

# Prever probabilidade calibrada
df_leads["probabilidade_conversao_calibrada"] = modelo_calibrado.predict_proba(X_calibrado)[:, 1].round(6)

# Score híbrido = média do total normalizado + probabilidade calibrada
scaler = MinMaxScaler()
df_leads["score_normalizado"] = scaler.fit_transform(df_leads[["leadscore_estimado_total"]])
df_leads["score_hibrido"] = ((df_leads["score_normalizado"] + df_leads["probabilidade_conversao_calibrada"]) / 2).round(4)

# Marcar como possível comprador se faixa A ou B
df_leads["possivel_comprador_regressivo"] = df_leads["faixa_predita_por_regressao"].isin(["A", "B"])

# Converter para porcentagem e formatar com símbolo %
df_leads["probabilidade_conversao_calibrada"] = (df_leads["probabilidade_conversao_calibrada"] * 100).round(0).astype(int).astype(str) + "%"
df_leads["score_hibrido"] = (df_leads["score_hibrido"] * 100).round(0).astype(int).astype(str) + "%"

df_leads.sort_values("leadscore_estimado_total", ascending=False).head(10)

,email,genero,faixa_etaria,escolaridade_categoria,renda_media,profissao_categoria,onde_acompanha_conteudo,nivel_idioma,fala_outro_idioma_categoria,motivo_fluencia_espanhol_categoria,...,tempo_antes_redes_sociais,fez_curso_espanhol,ouviu_falar_portal_vhe,lancamentos,leadscore_estimado_total,faixa_predita_por_regressao,probabilidade_conversao_calibrada,score_normalizado,score_hibrido,possivel_comprador_regressivo
3456,lead3456@exemplo.com,Feminino,35 - 44,Pós-graduação completa,De 5.000 a 10.000,Administrativo,YouTube,Iniciante (voltando),Não Fala Outro Idioma,Viagem,...,>6 meses,Sim,Sim,L2-25,4684.51,A,22%,1.000000,61%,True
3540,lead3540@exemplo.com,Feminino,25 - 34,Pós-graduação completa,De 2.500 a 5.000,Tecnologia,Outros,Iniciante (voltando),Não Fala Outro Idioma,Viagem,...,<1 mês,Não,Não,L2-25,4625.26,A,18%,0.979141,58%,True
994,lead994@exemplo.com,Feminino,25 - 34,Pós-graduação completa,De 5.000 a 10.000,Vendas,Facebook,Intermediário,Não Fala Outro Idioma,Viagem,...,3-6 meses,Não,Sim,L2-25,4621.26,A,13%,0.977733,55%,True
397,lead397@exemplo.com,Feminino,18 - 24,Pós-graduação completa,De 2.500 a 5.000,Vendas,YouTube,Intermediário,Não Fala Outro Idioma,Viagem,...,<1 mês,Sim,Sim,L2-25,4606.82,A,44%,0.972649,71%,True
3627,lead3627@exemplo.com,Feminino,18 - 24,Pós-graduação completa,De 2.500 a 5.000,Vendas,YouTube,Intermediário,Não Fala Outro Idioma,Todas Alternativas Acima,...,3-6 meses,Não,Sim,L2-25,4594.12,A,24%,0.968178,60%,True
2761,lead2761@exemplo.com,Feminino,18 - 24,Pós-graduação completa,De 2.500 a 5.000,Docência,Facebook,Iniciante (voltando),Não Fala Outro Idioma,Viagem,...,3-6 meses,Sim,Não,L2-25,4578.88,A,21%,0.962813,59%,True
2372,lead2372@exemplo.com,Feminino,25 - 34,Pós-graduação completa,De 5.000 a 10.000,Vendas,YouTube,Iniciante (voltando),Não Fala Outro Idioma,Pessoal,...,<1 mês,Não,Não,L2-25,4566.76,A,20%,0.958546,58%,True
1123,lead1123@exemplo.com,Feminino,+55,Pós-graduação completa,De 2.500 a 5.000,Docência,Instagram,Intermediário,Não Fala Outro Idioma,Pessoal,...,>6 meses,Não,Não,L2-25,4564.82,A,11%,0.957863,53%,True
3873,lead3873@exemplo.com,Feminino,25 - 34,Pós-graduação completa,De 5.000 a 10.000,CLT,Instagram,Iniciante (voltando),Não Fala Outro Idioma,Trabalho,...,<1 mês,Não,Sim,L2-25,4553.72,A,17%,0.953956,56%,True
1561,lead1561@exemplo.com,Feminino,44 - 55,Pós-graduação completa,De 5.000 a 10.000,Vendas,Outros,Intermediário,Não Fala Outro Idioma,Intercâmbio,...,>6 meses,Sim,Não,L2-25,4543.22,A,48%,0.950259,72%,True


In [2]:
print("Média compradores:", limites["media_compradores"])
print("Limite A:", limites["limite_a"])
print("Limite B:", limites["limite_b"])
print("Limite C:", limites["limite_c"])
print("Limite D:", limites["limite_d"])

# Ver faixa de valores previstos
print("\nMin score previsto:", df_leads["leadscore_estimado_total"].min())
print("Max score previsto:", df_leads["leadscore_estimado_total"].max())

Média compradores: 4200.91452991453
Limite A: 0.7220139784946238
Limite B: 0.5907387096774194
Limite C: 0.459463440860215
Limite D: 2100.457264957265

Min score previsto: 1843.99
Max score previsto: 4684.51


In [3]:
df_leads.to_csv(r"C:\Users\Camilo_Bica\data_science\consultoria\portal_vhe\dados\leads_monitorados.csv", index=False)
print("Arquivo 'leads_novos_classificados.csv' salvo com sucesso.")

Arquivo 'leads_novos_classificados.csv' salvo com sucesso.


In [4]:
df_leads["faixa_predita_por_regressao"].value_counts(normalize=True).round(3) * 100

faixa_predita_por_regressao
A    100.0
Name: proportion, dtype: float64